In [1]:
import sys

import pandas as pd
import numpy as np

In [2]:
# 판다스 보기 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
# csv 파일 로드
df_case = pd.read_csv('../datas/covid_worldwide.csv')
df_wid = pd.read_csv('../datas/외교부_국가(지역)별 일반현황_20220629.csv', encoding='cp949')
df_world_map=pd.read_excel('../datas/worldcities.xlsx',engine='openpyxl')

In [4]:
df23 = df_case.sort_values(by ='Population')
df23

,Serial Number,Country,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Test,Population
189,190,Djibouti,"15,690",189,"15,427",74,"305,941","1,016,097"
139,140,Eswatini,"74,133","1,422","72,676",35,"1,048,704","1,184,817"
82,83,Cyprus,"642,663","1,287","633,422","7,954","9,640,118","1,223,387"
158,159,Mauritius,"41,652","1,043","39,943",666,"358,675","1,274,727"
85,86,Estonia,"613,891","2,912","524,990","85,989","3,659,157","1,321,910"
174,175,Timor-Leste,"23,415",138,"23,102",175,"278,529","1,369,429"
230,231,Tokelau,5,NaN,NaN,5,NaN,"1,378"
118,119,Trinidad and Tobago,"187,908","4,326","183,264",318,"888,936","1,406,585"
1,2,India,"44,682,784","530,740","44,150,289","1,755","915,265,788","1,406,631,776"
184,185,Equatorial Guinea,"17,190",183,"16,883",124,"365,697","1,496,662"


In [5]:
# 위도 경도 파일 로드
df_world_map=df_world_map[['country','lat','lng']]
df_world_map=df_world_map.drop_duplicates(['country'])
df_world_map=df_world_map.reset_index(drop=True)
df_world_map=df_world_map.sort_values('country')
df_world_map=df_world_map.rename(columns={'country':'국가명', 'lat' : '위도', 'lng': '경도'})
df_world_map=df_world_map.sort_values('국가명')

In [6]:
# 코로나 집계 데이터 프레임 열 인덱스 재설정
dfc0 = df_case.rename(columns={'Serial Number': '고유번호', 'Country' : '국가명', 'Total Cases' : '확진수',
                        'Total Deaths' : '사망수', 'Total Recovered' : '회복수',
                        'Active Cases': '현발병', 'Total Test': '검사수', 'Population' : '인구'})
print(dfc0.columns)

Index(['고유번호', '국가명', '확진수', '사망수', '회복수', '현발병', '검사수', '인구'], dtype='object')


In [7]:
# 필요없는 자료열 제거
dfc0.drop(['고유번호','현발병'], axis=1, inplace=True)

In [8]:
# 정수형 변환을 위해 특수문자 제거, 인덱스 설정, NaN값 자료행 제거
dfc0 = dfc0.replace(',','',regex=True) 
dfc0 = dfc0.set_index(['국가명'])

dfc=dfc0.dropna()

In [9]:
# 각 요소를 정수형으로 변환
for i in dfc.columns:
    dfc[i]=dfc[i].astype('int')

C:\Users\82104\AppData\Local\Temp\ipykernel_10436\1622070333.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc[i]=dfc[i].astype('int')


In [10]:
# 자료숫자가 매우 적은 행(나라) 제거
dfc.drop(dfc[dfc['인구']  < (dfc['인구'].mean()*0.05)].index, inplace=True)

C:\Users\82104\AppData\Local\Temp\ipykernel_10436\3091841143.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.drop(dfc[dfc['인구']  < (dfc['인구'].mean()*0.05)].index, inplace=True)


In [11]:
dfc

,확진수,사망수,회복수,검사수,인구
국가명,,,,,
USA,104196861,1132935,101322779,1159832679,334805269
India,44682784,530740,44150289,915265788,1406631776
France,39524311,164233,39264546,271490188,65584518
Germany,37779833,165711,37398100,122332384,83883596
Brazil,36824580,697074,35919372,63776166,215353593
Japan,32588442,68399,21567425,92144639,125584838
S. Korea,30197066,33486,29740877,15804065,51329899
Italy,25453789,186833,25014986,265478247,60262770
UK,24274361,204171,24020088,522526476,68497907


In [12]:
dfc_fin = dfc.copy()
dfc_fin
# 면적 자료 데이터프레임 설정 및 가공
dw1= df_wid[['영문 국가명','면적']]
dw1=dw1.rename(columns={'영문 국가명' : '국가명'})
dw1
# merge 위해 위의 자료의 인덱스를 다시 열로 설정
dfc_fin=dfc_fin.reset_index()
dfc_fin

,국가명,확진수,사망수,회복수,검사수,인구
0,USA,104196861,1132935,101322779,1159832679,334805269
1,India,44682784,530740,44150289,915265788,1406631776
2,France,39524311,164233,39264546,271490188,65584518
3,Germany,37779833,165711,37398100,122332384,83883596
4,Brazil,36824580,697074,35919372,63776166,215353593
5,Japan,32588442,68399,21567425,92144639,125584838
6,S. Korea,30197066,33486,29740877,15804065,51329899
7,Italy,25453789,186833,25014986,265478247,60262770
8,UK,24274361,204171,24020088,522526476,68497907
9,Russia,21958696,395108,21356008,273400000,145805947


In [13]:
# 면적과 위경도 데이터프레임 일괄 병합
dft = pd.merge(dfc_fin,df_world_map,how='outer',on='국가명')
dft = dft.set_index(['국가명'])
dft = pd.merge(dft,dw1, how ='outer', on = '국가명')
dft = dft.set_index(['국가명'])
dft

,확진수,사망수,회복수,검사수,인구,위도,경도,면적
국가명,,,,,,,,
USA,104196861.00,1132935.00,101322779.00,1159832679.00,334805269.00,NaN,NaN,NaN
India,44682784.00,530740.00,44150289.00,915265788.00,1406631776.00,28.67,77.22,3287782.00
France,39524311.00,164233.00,39264546.00,271490188.00,65584518.00,48.86,2.35,675417.00
Germany,37779833.00,165711.00,37398100.00,122332384.00,83883596.00,52.52,13.38,357580.00
Brazil,36824580.00,697074.00,35919372.00,63776166.00,215353593.00,-23.55,-46.63,8510000.00
Japan,32588442.00,68399.00,21567425.00,92144639.00,125584838.00,35.68,139.77,378000.00
S. Korea,30197066.00,33486.00,29740877.00,15804065.00,51329899.00,NaN,NaN,NaN
Italy,25453789.00,186833.00,25014986.00,265478247.00,60262770.00,41.89,12.48,NaN
UK,24274361.00,204171.00,24020088.00,522526476.00,68497907.00,NaN,NaN,NaN


In [14]:
# 일부 주요나라 누락자료(면적) 입력을 위한 딕셔너리 생성 - 수작업
C = {'UAE': 83600 , 'Hong Kong': 1114, 'UK':243610 , 'Czechia':78867, 'Italy':302072, 'S. Korea': 100210,
     'USA': 9830000,'Papua New Guinea': 452860, 'CAR' : 623000, 'DRC':2345408, 'Burundi' :  27834}

In [15]:
# 누락된 면적 요소 삽입
for a in dft.index:
    if a in C:
        dft.loc[a]['면적'] = C[a]

In [16]:
# 국가별 누락 데이터 딕셔너리 (위경도)
dict_loc = {'UAE':[23.7139,54.3035], 'CAR': [6.5741,20.4869], 'DRC' : [-2.6046, 22.2650], 'Congo':[-2.9244, 23.7446],
            'Gambia' :[13.4363, -15.4027],'North Macedonia' : [41.6505, 21.6511], 'USA' :[39.5265, -102.0569],
            'UK' :[54.9817, -2.8012], 'S. Korea':[37.5666, 126.9780]}

In [17]:
# 위경도 요소 삽입
for nat in dft.index:
    if nat in dict_loc:
        dft.loc[nat]['위도'] = dict_loc[nat][0]
        dft.loc[nat]['경도'] = dict_loc[nat][1]
        
dft

,확진수,사망수,회복수,검사수,인구,위도,경도,면적
국가명,,,,,,,,
USA,104196861.00,1132935.00,101322779.00,1159832679.00,334805269.00,39.53,-102.06,9830000.00
India,44682784.00,530740.00,44150289.00,915265788.00,1406631776.00,28.67,77.22,3287782.00
France,39524311.00,164233.00,39264546.00,271490188.00,65584518.00,48.86,2.35,675417.00
Germany,37779833.00,165711.00,37398100.00,122332384.00,83883596.00,52.52,13.38,357580.00
Brazil,36824580.00,697074.00,35919372.00,63776166.00,215353593.00,-23.55,-46.63,8510000.00
Japan,32588442.00,68399.00,21567425.00,92144639.00,125584838.00,35.68,139.77,378000.00
S. Korea,30197066.00,33486.00,29740877.00,15804065.00,51329899.00,37.57,126.98,100210.00
Italy,25453789.00,186833.00,25014986.00,265478247.00,60262770.00,41.89,12.48,302072.00
UK,24274361.00,204171.00,24020088.00,522526476.00,68497907.00,54.98,-2.80,243610.00


In [18]:
# 누락데이터 행 제거
df_tot = dft.dropna()

In [19]:
df_tot

,확진수,사망수,회복수,검사수,인구,위도,경도,면적
국가명,,,,,,,,
USA,104196861.00,1132935.00,101322779.00,1159832679.00,334805269.00,39.53,-102.06,9830000.00
India,44682784.00,530740.00,44150289.00,915265788.00,1406631776.00,28.67,77.22,3287782.00
France,39524311.00,164233.00,39264546.00,271490188.00,65584518.00,48.86,2.35,675417.00
Germany,37779833.00,165711.00,37398100.00,122332384.00,83883596.00,52.52,13.38,357580.00
Brazil,36824580.00,697074.00,35919372.00,63776166.00,215353593.00,-23.55,-46.63,8510000.00
Japan,32588442.00,68399.00,21567425.00,92144639.00,125584838.00,35.68,139.77,378000.00
S. Korea,30197066.00,33486.00,29740877.00,15804065.00,51329899.00,37.57,126.98,100210.00
Italy,25453789.00,186833.00,25014986.00,265478247.00,60262770.00,41.89,12.48,302072.00
UK,24274361.00,204171.00,24020088.00,522526476.00,68497907.00,54.98,-2.80,243610.00


In [20]:
# 확진률 (확진수/검사수)
# 사망률 (사망수/확진수)
# 회복률 (회복수/확진수)
# 검사율 (검사수/인구)
# 열 추가 함수

col_dict = {'확진수':'검사수', '사망수':'확진수', '회복수': '확진수', '검사수':'인구'}

for j in col_dict:
    j = str(j)
    T = []
    for i in df_tot.index:
        i = str(i)
        T.append(df_tot.loc[i,j]/df_tot.loc[i,str(col_dict[j])]*100)
    df_tot[f'{j}율']= T
        

C:\Users\82104\AppData\Local\Temp\ipykernel_10436\3386712382.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tot[f'{j}율']= T
C:\Users\82104\AppData\Local\Temp\ipykernel_10436\3386712382.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tot[f'{j}율']= T
C:\Users\82104\AppData\Local\Temp\ipykernel_10436\3386712382.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [21]:
# 정렬
df_tot = df_tot.sort_values(by='검사수율', ascending=False)
 

In [22]:
def pop_den(df):  # 인구밀도 열 추가 함수
    T =[]
    for i in df.index:
        T.append(df.loc[i]['인구']/df.loc[i]['면적'])
    df['인구밀도'] = T
    
def pop_par(df):  # 감염율 열 추가 함수
    P =[]
    for i in df.index:
        P.append(df.loc[i]['확진수']/df.loc[i]['인구']*100)
    df['감염율'] = P

In [23]:
pop_den(df_tot)
pop_par(df_tot)
df_tot

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율,사망수율,회복수율,검사수율,인구밀도,감염율
국가명,,,,,,,,,,,,,,
Austria,5780229.00,21689.00,5730189.00,211273524.00,9066710.00,48.21,16.37,83879.00,2.74,0.38,99.13,2330.21,108.09,63.75
Denmark,3173247.00,8145.00,3163849.00,129144754.00,5834950.00,55.68,12.56,42934.00,2.46,0.26,99.70,2213.30,135.91,54.38
UAE,1049409.00,2348.00,1032735.00,198638806.00,10081785.00,23.71,54.30,83600.00,0.53,0.22,98.41,1970.27,120.60,10.41
Spain,13731478.00,118434.00,13557699.00,471036328.00,46719142.00,40.42,-3.72,505370.00,2.92,0.86,98.73,1008.23,92.45,29.39
Hong Kong,2877280.00,13358.00,2509483.00,76123870.00,7604299.00,22.31,114.18,1114.00,3.78,0.46,87.22,1001.06,6826.12,37.84
Greece,5708301.00,35630.00,5662212.00,102228365.00,10316637.00,37.98,23.73,131957.00,5.58,0.62,99.19,990.91,78.18,55.33
UK,24274361.00,204171.00,24020088.00,522526476.00,68497907.00,54.98,-2.80,243610.00,4.65,0.84,98.95,762.84,281.18,35.44
Bahrain,700452.00,1543.00,698400.00,10682444.00,1783983.00,26.23,50.58,765.00,6.56,0.22,99.71,598.80,2332.00,39.26
Czechia,4590019.00,42312.00,4538304.00,56893223.00,10736784.00,50.08,14.42,78867.00,8.07,0.92,98.87,529.89,136.14,42.75


In [24]:
df_tot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132 entries, Austria to Algeria
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   확진수     132 non-null    float64
 1   사망수     132 non-null    float64
 2   회복수     132 non-null    float64
 3   검사수     132 non-null    float64
 4   인구      132 non-null    float64
 5   위도      132 non-null    float64
 6   경도      132 non-null    float64
 7   면적      132 non-null    float64
 8   확진수율    132 non-null    float64
 9   사망수율    132 non-null    float64
 10  회복수율    132 non-null    float64
 11  검사수율    132 non-null    float64
 12  인구밀도    132 non-null    float64
 13  감염율     132 non-null    float64
dtypes: float64(14)
memory usage: 19.5+ KB


In [25]:
df_tot.loc['총합'] = df_tot.iloc[:130,:5].sum()
df_tot.loc['평균'] = df_tot.iloc[:130,:].mean()

In [26]:
df_tot.head()

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율,사망수율,회복수율,검사수율,인구밀도,감염율
국가명,,,,,,,,,,,,,,
Austria,5780229.00,21689.00,5730189.00,211273524.00,9066710.00,48.21,16.37,83879.00,2.74,0.38,99.13,2330.21,108.09,63.75
Denmark,3173247.00,8145.00,3163849.00,129144754.00,5834950.00,55.68,12.56,42934.00,2.46,0.26,99.70,2213.30,135.91,54.38
UAE,1049409.00,2348.00,1032735.00,198638806.00,10081785.00,23.71,54.30,83600.00,0.53,0.22,98.41,1970.27,120.60,10.41
Spain,13731478.00,118434.00,13557699.00,471036328.00,46719142.00,40.42,-3.72,505370.00,2.92,0.86,98.73,1008.23,92.45,29.39
Hong Kong,2877280.00,13358.00,2509483.00,76123870.00,7604299.00,22.31,114.18,1114.00,3.78,0.46,87.22,1001.06,6826.12,37.84


In [27]:
df_data=df_tot.copy()
df_data.drop(['총합','평균'],axis=0,inplace=True)

In [28]:
df_data

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율,사망수율,회복수율,검사수율,인구밀도,감염율
국가명,,,,,,,,,,,,,,
Austria,5780229.00,21689.00,5730189.00,211273524.00,9066710.00,48.21,16.37,83879.00,2.74,0.38,99.13,2330.21,108.09,63.75
Denmark,3173247.00,8145.00,3163849.00,129144754.00,5834950.00,55.68,12.56,42934.00,2.46,0.26,99.70,2213.30,135.91,54.38
UAE,1049409.00,2348.00,1032735.00,198638806.00,10081785.00,23.71,54.30,83600.00,0.53,0.22,98.41,1970.27,120.60,10.41
Spain,13731478.00,118434.00,13557699.00,471036328.00,46719142.00,40.42,-3.72,505370.00,2.92,0.86,98.73,1008.23,92.45,29.39
Hong Kong,2877280.00,13358.00,2509483.00,76123870.00,7604299.00,22.31,114.18,1114.00,3.78,0.46,87.22,1001.06,6826.12,37.84
Greece,5708301.00,35630.00,5662212.00,102228365.00,10316637.00,37.98,23.73,131957.00,5.58,0.62,99.19,990.91,78.18,55.33
UK,24274361.00,204171.00,24020088.00,522526476.00,68497907.00,54.98,-2.80,243610.00,4.65,0.84,98.95,762.84,281.18,35.44
Bahrain,700452.00,1543.00,698400.00,10682444.00,1783983.00,26.23,50.58,765.00,6.56,0.22,99.71,598.80,2332.00,39.26
Czechia,4590019.00,42312.00,4538304.00,56893223.00,10736784.00,50.08,14.42,78867.00,8.07,0.92,98.87,529.89,136.14,42.75


In [29]:
df_data.sort_values('확진수율')

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율,사망수율,회복수율,검사수율,인구밀도,감염율
국가명,,,,,,,,,,,,,,
UAE,1049409.00,2348.00,1032735.00,198638806.00,10081785.00,23.71,54.30,83600.00,0.53,0.22,98.41,1970.27,120.60,10.41
Oman,399449.00,4628.00,384669.00,25000000.00,5323993.00,23.61,58.59,309500.00,1.60,1.16,96.30,469.57,17.20,7.50
Saudi Arabia,827962.00,9574.00,815528.00,45048008.00,35844909.00,24.65,46.71,2150000.00,1.84,1.16,98.50,125.67,16.67,2.31
Rwanda,133090.00,1468.00,131559.00,5959042.00,13600464.00,-1.95,30.06,26338.00,2.23,1.10,98.85,43.81,516.38,0.98
Denmark,3173247.00,8145.00,3163849.00,129144754.00,5834950.00,55.68,12.56,42934.00,2.46,0.26,99.70,2213.30,135.91,54.38
Austria,5780229.00,21689.00,5730189.00,211273524.00,9066710.00,48.21,16.37,83879.00,2.74,0.38,99.13,2330.21,108.09,63.75
Spain,13731478.00,118434.00,13557699.00,471036328.00,46719142.00,40.42,-3.72,505370.00,2.92,0.86,98.73,1008.23,92.45,29.39
Gabon,48981.00,306.00,48674.00,1621909.00,2331533.00,0.39,9.45,267000.00,3.02,0.62,99.37,69.56,8.73,2.10
Yemen,11945.00,2159.00,9124.00,329592.00,31154867.00,15.35,44.20,527968.00,3.62,18.07,76.38,1.06,59.01,0.04


In [30]:
df_data.sort_values('사망수율')

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율,사망수율,회복수율,검사수율,인구밀도,감염율
국가명,,,,,,,,,,,,,,
Burundi,53492.00,38.00,53394.00,345742.00,12624840.00,-3.38,29.36,27834.00,15.47,0.07,99.82,2.74,453.58,0.42
Singapore,2217110.00,1722.00,2144490.00,24756666.00,5943546.00,1.30,103.80,719.00,8.96,0.08,96.72,416.53,8266.41,37.30
S. Korea,30197066.00,33486.00,29740877.00,15804065.00,51329899.00,37.57,126.98,100210.00,191.07,0.11,98.49,30.79,512.22,58.83
Qatar,492534.00,686.00,491585.00,4065369.00,2979915.00,25.30,51.53,11581.00,12.12,0.14,99.81,136.43,257.31,16.53
Australia,11295446.00,18615.00,11235771.00,78835048.00,26068792.00,-33.87,151.21,7690000.00,14.33,0.16,99.47,302.41,3.39,43.33
Taiwan,9569611.00,16356.00,9129766.00,30207485.00,23888595.00,24.15,120.67,36000.00,31.68,0.17,95.40,126.45,663.57,40.06
New Zealand,2182355.00,3781.00,2169320.00,7710637.00,4898203.00,-36.85,174.78,270000.00,28.30,0.17,99.40,157.42,18.14,44.55
Japan,32588442.00,68399.00,21567425.00,92144639.00,125584838.00,35.68,139.77,378000.00,35.37,0.21,66.18,73.37,332.24,25.95
Mongolia,1007871.00,2179.00,1005658.00,4030048.00,3378078.00,47.92,106.91,1564000.00,25.01,0.22,99.78,119.30,2.16,29.84


In [31]:
df_data.sort_values('검사수율')

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율,사망수율,회복수율,검사수율,인구밀도,감염율
국가명,,,,,,,,,,,,,,
Algeria,271378.00,6881.00,182749.00,230861.00,45350148.00,36.78,3.06,2381741.00,117.55,2.54,67.34,0.51,19.04,0.60
Eritrea,10189.00,103.00,10086.00,23693.00,3662244.00,15.33,38.92,117600.00,43.00,1.01,98.99,0.65,31.14,0.28
Syria,57453.00,3164.00,54289.00,146269.00,19364809.00,36.20,37.15,185180.00,39.28,5.51,94.49,0.76,104.57,0.30
DRC,95514.00,1463.00,84264.00,846704.00,95240792.00,-2.60,22.27,2345408.00,11.28,1.53,88.22,0.89,40.61,0.10
Niger,9931.00,312.00,8890.00,254538.00,26083660.00,13.51,2.11,1267000.00,3.90,3.14,89.52,0.98,20.59,0.04
Yemen,11945.00,2159.00,9124.00,329592.00,31154867.00,15.35,44.20,527968.00,3.62,18.07,76.38,1.06,59.01,0.04
Chad,7652.00,194.00,4874.00,191341.00,17413580.00,12.11,15.05,1284000.00,4.00,2.54,63.70,1.10,13.56,0.04
Burkina Faso,22025.00,396.00,21596.00,248995.00,22102838.00,12.37,-1.53,274200.00,8.85,1.80,98.05,1.13,80.61,0.10
Haiti,34076.00,860.00,33033.00,132422.00,11680283.00,18.54,-72.34,27750.00,25.73,2.52,96.94,1.13,420.91,0.29


In [32]:
df_data.sort_values('인구밀도')

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율,사망수율,회복수율,검사수율,인구밀도,감염율
국가명,,,,,,,,,,,,,,
Mongolia,1007871.00,2179.00,1005658.00,4030048.00,3378078.00,47.92,106.91,1564000.00,25.01,0.22,99.78,119.30,2.16,29.84
Namibia,170741.00,4088.00,166452.00,1062663.00,2633874.00,-22.57,17.08,824269.00,16.07,2.39,97.49,40.35,3.20,6.48
Venezuela,551587.00,5845.00,545003.00,3359014.00,29266991.00,10.50,-66.93,9120505.00,16.42,1.06,98.81,11.48,3.21,1.88
Australia,11295446.00,18615.00,11235771.00,78835048.00,26068792.00,-33.87,151.21,7690000.00,14.33,0.16,99.47,302.41,3.39,43.33
Canada,4550256.00,50380.00,4444013.00,66343123.00,38388419.00,43.74,-79.37,9980000.00,6.86,1.11,97.67,172.82,3.85,11.85
Libya,507162.00,6437.00,500721.00,2483446.00,7040745.00,32.88,13.19,1759540.00,20.42,1.27,98.73,35.27,4.00,7.20
Botswana,329227.00,2800.00,325934.00,2026898.00,2441162.00,-24.66,25.91,582000.00,16.24,0.85,99.00,83.03,4.19,13.49
Mauritania,63665.00,997.00,62437.00,1009957.00,4901981.00,18.09,-15.98,1030700.00,6.30,1.57,98.07,20.60,4.76,1.30
Kazakhstan,1405917.00,13695.00,1383020.00,11575012.00,19205043.00,43.25,76.90,2724900.00,12.15,0.97,98.37,60.27,7.05,7.32


In [33]:
df_data.사망수율.describe()

count   132.00
mean      1.63
std       1.89
min       0.07
25%       0.72
50%       1.19
75%       2.08
max      18.07
Name: 사망수율, dtype: float64